In [5]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')

# VRP

In [16]:
# This part!!
# ============================================================
# 0)  CONFIG
# ============================================================
FILE = "new_matrix.csv"          # distance/time matrix (seconds)
TIME_LIMIT_SEC = 60             # give the solver 3 minutes
MAX_STOPS_PER_ROUTE = 8      # None = unlimited

# exact list & order of depots to use
TERMINALS = ['BD14', 'BD15', 'BT03', 'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']

# ============================================================
# 1)  LOAD & CLEAN THE MATRIX
# ============================================================
import pandas as pd, numpy as np
!pip install ortools
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

df  = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/new_matrix.csv', index_col=0)                 # IDs in the index
bad = df.apply(pd.to_numeric, errors="coerce")   # force numeric

if bad.isna().any().any():
    r = bad.index[ bad.isna().any(axis=1) ][0]
    c = bad.columns[ bad.isna().any(axis=0) ][0]
    raise ValueError(f"non-numeric cell at row {r!r}, column {c!r}")

dist_mat = bad.astype(np.int32).to_numpy()       # seconds
ids      = df.index.astype(str).tolist()
n_nodes  = len(ids)

# ------------------------------------------------------------
# make sure every requested depot exists in the CSV
missing = [d for d in TERMINALS if d not in ids]
assert not missing, f"Depot(s) not found in CSV: {missing}"

# OR-Tools wants the depot indices
bus_idx = [ids.index(d) for d in TERMINALS]

# classify the other nodes (not strictly needed, but sanity)
acc_idx   = [i for i,s in enumerate(ids) if s.startswith("A")]
venue_idx = [i for i,s in enumerate(ids) if s.startswith("V")]
assert acc_idx and venue_idx, "Need at least one A- and one V- node"

# ============================================================
# 2)  ROUTING MODEL
# ============================================================
man = pywrapcp.RoutingIndexManager(n_nodes, len(bus_idx), bus_idx, bus_idx)
rt  = pywrapcp.RoutingModel(man)

# transit (cost) = travel seconds
def sec_cb(fi, ti):
    f, t = man.IndexToNode(fi), man.IndexToNode(ti)
    return int(dist_mat[f][t])

transit = rt.RegisterTransitCallback(sec_cb)
rt.SetArcCostEvaluatorOfAllVehicles(transit)

# limit total stops if desired
if MAX_STOPS_PER_ROUTE:
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, MAX_STOPS_PER_ROUTE, True, "Stops")

# ---- force every bus to be used (≥ 1 real stop) ---------------------------
if MAX_STOPS_PER_ROUTE:
    stop_dim = rt.GetDimensionOrDie("Stops")
else:
    # create a tiny 1-per-hop dimension just for this purpose
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, 10**6, True, "MustUse")
    stop_dim = rt.GetDimensionOrDie("MustUse")


# ============================================================
# 3)  SEARCH PARAMETERS
# ============================================================
p = pywrapcp.DefaultRoutingSearchParameters()
p.first_solution_strategy    = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
p.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
p.time_limit.seconds         = TIME_LIMIT_SEC

solution = rt.SolveWithParameters(p)

# ============================================================
# 4)  SINGLE-LINE REPORT  (seconds → minutes)
# ============================================================
if not solution:
    print("❌  No solution found — try increasing TIME_LIMIT_SEC "
          "or relaxing MAX_STOPS_PER_ROUTE.")
else:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route = [ids[depot_node]]          # start depot
        time_sec = 0

        while not rt.IsEnd(idx):
            prev = idx
            idx  = solution.Value(rt.NextVar(idx))
            route.append(ids[man.IndexToNode(idx)])
            time_sec += dist_mat[man.IndexToNode(prev), man.IndexToNode(idx)]

        stops = len(route) - 2              # minus start & return depot
        minutes = time_sec / 60
        print(f"{ids[depot_node]:<5}: " + " → ".join(route)
              + f"   ({stops} stops, {minutes:.1f} min)")

BD14 : BD14 → A12 → V17 → A33 → A34 → A14 → V31 → A13 → BD14   (7 stops, 72.0 min)
BD15 : BD15 → A1 → A21 → A4 → A3 → A2 → BD15   (5 stops, 58.5 min)
BT03 : BT03 → A35 → BD16 → A32 → A11 → A36 → A45 → A16 → BT03   (7 stops, 163.1 min)
BL14 : BL14 → A15 → A40 → V30 → A18 → A8 → BT24 → BL14   (6 stops, 103.3 min)
BD04 : BD04 → A17 → V9 → BD04   (2 stops, 26.6 min)
BT11 : BT11 → A46 → A23 → BT16 → A43 → V24 → V14 → A31 → BT11   (7 stops, 73.7 min)
BT06 : BT06 → A28 → BT08 → A41 → A27 → V5 → A29 → A26 → BT06   (7 stops, 68.8 min)
BT21 : BT21 → A19 → A20 → V28 → A44 → A37 → V18 → A7 → BT21   (7 stops, 97.8 min)
BL20 : BL20 → A51 → V8 → V11 → A50 → A47 → A48 → A49 → BL20   (7 stops, 60.3 min)
BT18 : BT18 → A24 → A9 → A30 → A38 → A42 → V20 → A22 → BT18   (7 stops, 228.9 min)
BL23 : BL23 → A39 → A10 → V12 → A6 → A25 → V19 → A5 → BL23   (7 stops, 115.3 min)


### node_ids

In [7]:
# Extract the first column (index) from new_matrix
node_ids = df.index.tolist()

# Initialize an empty list to store the coordinates
coordinates = []

# Iterate through the node IDs and match them with the corresponding coordinates
for node_id in node_ids:
    if node_id in venues['id'].values:
        # Match with venues
        venue_row = venues[venues['id'] == node_id]
        coordinates.append([node_id, venue_row['Latitude'].values[0], venue_row['Longitude'].values[0]])
    elif node_id in accomodations_clusters['id'].values:
        # Match with accommodation clusters
        accom_row = accomodations_clusters[accomodations_clusters['id'] == node_id]
        coordinates.append([node_id, accom_row['avg_latitude'].values[0], accom_row['avg_longitude'].values[0]])
    elif node_id in bus_terminals['id'].values:
        # Match with bus terminals
        terminal_row = bus_terminals[bus_terminals['id'] == node_id]
        coordinates.append([node_id, terminal_row['Latitude'].values[0], terminal_row['Longitude'].values[0]])
    else:
        # If no match is found, append NaN for coordinates
        coordinates.append([node_id, float('nan'), float('nan')])

# Create the node_coords dataframe
node_coords = pd.DataFrame(coordinates, columns=['Node ID', 'Latitude', 'Longitude'])

# Display the resulting dataframe
node_coords

,Node ID,Latitude,Longitude
0,BD14,33.993948,-118.476437
1,BD15,34.085197,-118.382010
2,BD16,34.236880,-118.598039
3,BT16,34.041206,-118.187315
4,BT03,34.201350,-118.450750
...,...,...,...
75,A47,34.091356,-118.279988
76,A48,34.063247,-118.298355
77,A49,34.054311,-118.280299
78,A50,34.083647,-118.255568


In [8]:
# Add a new column 'Type' to node_coords based on the first letter of 'Node ID'
node_coords['Type'] = node_coords['Node ID'].apply(
    lambda x: 'Depot' if x.startswith('B') else 
              'Accommodation' if x.startswith('A') else 
              'Venue' if x.startswith('V') else 'Unknown'
)

# Display the updated dataframe
node_coords

,Node ID,Latitude,Longitude,Type
0,BD14,33.993948,-118.476437,Depot
1,BD15,34.085197,-118.382010,Depot
2,BD16,34.236880,-118.598039,Depot
3,BT16,34.041206,-118.187315,Depot
4,BT03,34.201350,-118.450750,Depot
...,...,...,...,...
75,A47,34.091356,-118.279988,Accommodation
76,A48,34.063247,-118.298355,Accommodation
77,A49,34.054311,-118.280299,Accommodation
78,A50,34.083647,-118.255568,Accommodation


In [9]:
node_ids = node_coords['Node ID'].tolist()
node_id_to_coords = dict(zip(node_coords['Node ID'], zip(node_coords['Latitude'], node_coords['Longitude'])))
node_id_to_type = dict(zip(node_coords['Node ID'], node_coords['Type']))


### map!

In [17]:
import folium
from folium import PolyLine


# Map center around LA
la_center = [34.0522, -118.2437]
m = folium.Map(location=la_center, zoom_start=11)

# Color mapping for node types
type_colors = {
    'Depot': 'red',
    'Venue': 'green',
    'Accommodation': 'blue'
}

# Emoji mapping for tooltip fun
type_emojis = {
    'Depot': '🚌',
    'Venue': '🏟️',
    'Accommodation': '🏨'
}

# Add markers for all nodes
for node_id in node_ids:
    coord = node_id_to_coords.get(node_id)
    if coord is None:
        continue  # skip if coordinates missing
    
    node_type = node_id_to_type.get(node_id, 'Accommodation')  # default to Accommodation
    color = type_colors.get(node_type, 'gray')
    emoji = type_emojis.get(node_type, '')
    
    folium.CircleMarker(
        location=coord,
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.9,
        tooltip=f"{emoji} {node_id} ({node_type})"
    ).add_to(m)

# Colors for routes
route_colors = ['purple', 'orange', 'darkred', 'cadetblue', 'darkblue', 'pink', 'gray', 'darkgreen']

# Plot the routes from your solver solution
if solution:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route_coords = []
        
        while not rt.IsEnd(idx):
            node = man.IndexToNode(idx)
            node_id = ids[node]
            coord = node_id_to_coords.get(node_id)
            if coord:
                route_coords.append(coord)
            idx = solution.Value(rt.NextVar(idx))
        
        # Add last depot coord to close the route
        end_node = man.IndexToNode(idx)
        end_node_id = ids[end_node]
        end_coord = node_id_to_coords.get(end_node_id)
        if end_coord:
            route_coords.append(end_coord)
        
        # Draw polyline for route if it has at least 2 points
        if len(route_coords) > 1:
            folium.PolyLine(
                route_coords,
                color=route_colors[v % len(route_colors)],
                weight=5,
                opacity=0.7,
                tooltip=f"Bus route {v+1} (Depot {ids[depot_node]})"
            ).add_to(m)
else:
    print("No solution found to plot.")

# Display the map (in Jupyter Notebook)
m


In [18]:
m.save('/Users/clarabottinelli/Downloads/route_mapV4.html')

In [14]:
# Extract nodes from bus route 3
route_3_nodes = []
bus_index = 2  # Bus route 3 corresponds to index 2

if solution:
    idx = rt.Start(bus_index)
    while not rt.IsEnd(idx):
        node = man.IndexToNode(idx)
        route_3_nodes.append(ids[node])
        idx = solution.Value(rt.NextVar(idx))
    # Add the last depot node to close the route
    route_3_nodes.append(ids[man.IndexToNode(idx)])

route_3_nodes

['BD16', 'A32', 'A11', 'A36', 'A45', 'A35', 'BD16']

In [15]:
# Extract nodes from bus route 5
route_5_nodes = []
bus_index = 4  # Bus route 5 corresponds to index 4

if solution:
    idx = rt.Start(bus_index)
    while not rt.IsEnd(idx):
        node = man.IndexToNode(idx)
        route_5_nodes.append(ids[node])
        idx = solution.Value(rt.NextVar(idx))
    # Add the last depot node to close the route
    route_5_nodes.append(ids[man.IndexToNode(idx)])

route_5_nodes

['BT03', 'A16', 'V20', 'A9', 'A42', 'A30', 'A38', 'A24', 'BT03']